In [ ]:
import psycopg2
import pandas as pd

In [ ]:
def connect_db():
    conn = psycopg2.connect("host=localhost dbname=smi user=postgres password=password")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    return conn, cur

In [ ]:
def select_data(tableName, limit):
    cur.execute("select * from {} limit {}".format(tableName, limit))
    row = cur.fetchall()
    for r in row:
        print(r)

In [ ]:
def fetch_data(tableName):
    query = "select * from {}".format(tableName)
    df = pd.read_sql(query, conn)
    return df

In [ ]:
def toNumbers(df, col):
    df[col] = df[col].replace({"K":"*1e3", "M":"*1e6"}, regex=True).map(pd.eval).astype(int)

In [ ]:
conn, cur = connect_db()

In [ ]:
ii_df = fetch_data("insta_influencer")
ii_df

In [ ]:
toNumbers(ii_df, 'followers')
toNumbers(ii_df, 'authentic_engagement')
toNumbers(ii_df, 'engagement_avg')

In [ ]:
ii_df = ii_df.replace('NaN', '')

In [86]:
ii_df.head(5)
#ii_df.dtypes

,username,name,followers,audience_country,authentic_engagement,engagement_avg,category_1,category_2
0,433,433,48500000,Spain,383100,637000,Sports with a ball,
1,__youngbae__,TAEYANG,12700000,Indonesia,478000,542300,Music,
2,_agentgirl_,НАСТЯ ИВЛЕЕВА,18800000,Russia,310800,377900,Shows,
3,_imyour_joy,Joy,13500000,Indonesia,1100000,1400000,Lifestyle,
4,_jeongjaehyun,Jaehyun,11100000,Indonesia,2500000,3100000,,


In [89]:
ii_df_cols = list(ii_df.columns.values)

In [90]:
ii_df.dtypes

username                object
name                    object
followers                int32
audience_country        object
authentic_engagement     int32
engagement_avg           int32
category_1              object
category_2              object
dtype: object

In [92]:
dtype_list = []
for d in ii_df.dtypes:
    if d == 'int32':
        dtype_list.append('float')
    else:
        dtype_list.append('varchar')
dtype_list

['varchar',
 'varchar',
 'float',
 'varchar',
 'float',
 'float',
 'varchar',
 'varchar']

In [97]:
ddlStmt = 'create table if not exists final_insta_influencer ('

for i in range(len(dtype_list)):
    ddlStmt = ddlStmt + '\n' + ii_df_cols[i] + ' ' + dtype_list[i] + ','

ddlStmt = ddlStmt[:-1] + ' );'

In [98]:
print(ddlStmt)

create table if not exists final_insta_influencer (
username varchar,
name varchar,
followers float,
audience_country varchar,
authentic_engagement float,
engagement_avg float,
category_1 varchar,
category_2 varchar );


In [106]:
cur.execute(ddlStmt)

In [112]:
ii_insert_stmt = ("INSERT INTO public.final_insta_influencer \
(username, name, followers, audience_country, authentic_engagement, engagement_avg, category_1, category_2) \
VALUES(%s, %s, %s, %s, %s, %s, %s, %s);")

In [113]:
for i, row in ii_df.iterrows():
    cur.execute(ii_insert_stmt, list(row))

In [114]:
cur.execute("select * from public.final_insta_influencer limit 15")
res = cur.fetchall()
for r in res:
    print(r)

('433', '433', 48500000.0, 'Spain', 383100.0, 637000.0, 'Sports with a ball', '')
('__youngbae__', 'TAEYANG', 12700000.0, 'Indonesia', 478000.0, 542300.0, 'Music', '')
('_agentgirl_', 'НАСТЯ ИВЛЕЕВА', 18800000.0, 'Russia', 310800.0, 377900.0, 'Shows', '')
('_imyour_joy', 'Joy', 13500000.0, 'Indonesia', 1100000.0, 1400000.0, 'Lifestyle', '')
('_jeongjaehyun', 'Jaehyun', 11100000.0, 'Indonesia', 2500000.0, 3100000.0, '', '')
('_mariahwasa', 'HWASA', 7900000.0, 'Brazil', 915100.0, 1200000.0, '', '')
('_rl9', 'Robert Lewandowski', 25000000.0, 'Poland', 588600.0, 749000.0, 'Sports with a ball', '')
('_seorina', '설인아 sᴇᴏʀɪɴᴀ', 3000000.0, 'South Korea', 803900.0, 997800.0, 'Lifestyle', 'Cinema & Actors/actresses')
('_shotaroo_', 'SHOTARO ショウタロウ', 4900000.0, 'Indonesia', 860100.0, 1100000.0, '', '')
('03_hu', '박지후', 4099999.0, 'South Korea', 711000.0, 872600.0, '', '')
('21savage', '', 13200000.0, 'United States', 263500.0, 462000.0, 'Music', '')
('333cyj333', 'Ars 🎗', 6500000.0, 'Tha